Upload input-output datasets from Monte Carlo simulations and perform sensitivity analysis and construct fast metamodels.

[Link to repository in Github](https://github.com/TorbenOestergaard/mc_sa_ml)


# Setup

This notebook relies on several scripts and includes an example data file with Monte Carlo simulation results. Therefore, the notebook cannot be run by itself and it must have access to these resources. Below, you'll see three options to provide this access. Select the one that fits you best.

## Option A: Public github notebook

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TorbenOestergaard/mc_sa_ml/blob/main/BPS_to_ML_github.ipynb)

The fastest way to get started is to work online in the public github notebook.  
The content in the github repository *mc_sa_ml* must be cloned to the current Colab session in order to access the scripts and the example file. 

- Requirements: No requirements
- Changes: Your changes will *not* be saved when closing the browser.

In [ ]:
!git clone https://github.com/TorbenOestergaard/mc_sa_ml.git

my_path = 'mc_sa_ml/'

## Option B: Google Drive

You may import the folder Github repository *mc_sa_ml* and copy it to your Google Drive folder, for example to the 'Colab Notebooks" subfolder which is created when you make your first Colab notebook.  

- Requirements: Google Colab must be allowed to access the content in your Drive folder.
- Changes: Your changes will automatically be saved in your local notebook and files will be saved to your Drive folder. 

In [ ]:
# from google.colab import drive 
# drive.mount('/content/drive')

# my_path = '/content/drive/MyDrive/Colab Notebooks/mc_sa_ml/'

## Option C: Local Python interpreter

Finally, you may clone the *mc_sa_ml* repository to your local desktop and run it using your own Python interpreter. However, you need to install a number of packages (see imports below) and set the `my_path` variable accordingly.

In [ ]:
# my_path = 'your_local_path_to_mc_sa_ml_folder'

## Import dependencies

In [ ]:
global y_train_pred
global y_train_full_pred
global y_valid_pred
global y_test_pred

global x_train_prepared
global x_train_full_prepared
global x_valid_prepared
global x_test_prepared

from os import path, makedirs
import sys
from google.colab import files
import io

sys.path.append(my_path)

import scripts as src
from scripts import transform_categorical_features, ordinal_decode_cat_hyperparameters, create_new_samples, print_R2_performance, make_predictions, sa_multiple, obtain_pdfs 
import SAtom2 as sa

# Data analysis and visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import clear_output 
from ipywidgets import Dropdown
from os import path, makedirs

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

# Upload your data

## Upload file

Run cell below to create upload button. Then select a file with Monte Carlo simulations inputs and outputs.

In [ ]:
uploaded = files.upload()

Run to load the file, if present, transform categorical features to numeric to enable sensitivity analysis and machine learning.

In [ ]:
try:
  filename_monte_carlo = list(uploaded.keys())[0]
except:
  filename_monte_carlo = 'test_high_school_i10_o3.tsv'
print(f'Filename:   {filename_monte_carlo}')

file_extension = filename_monte_carlo.split(".")[-1]
file_name_only = filename_monte_carlo[:-(len(file_extension)+1)]

if file_extension == 'xlsx':
  # Excel file. If data in specific sheet, add argument: sheet_name='Sheet1'
  XY_raw = pd.read_excel(my_path + filename_monte_carlo, header=0, engine="openpyxl", )
elif file_extension == 'csv' or file_extension == 'txt':
  XY_raw = pd.read_csv(my_path +  filename_monte_carlo, sep=None)
elif file_extension == 'tsv':
  XY_raw = pd.read_csv(my_path + filename_monte_carlo, sep='\t')
print(f'N rows:     {XY_raw.shape[0]} \nN columns:  {XY_raw.shape[1]}')

# Transform pcategorical features
XY = transform_categorical_features(XY_raw, verbose=False)

XY_raw.head()

## Describe columns (important)

Specify the number of inputs and outputs to enable the scripts to distinguish between input columns and output columns.

In [ ]:
n_inputs = 10 # Specify the number of input columns
n_outputs = 3 # Specify the number of output columns

N = XY.shape[0]
X = XY.iloc[:N, :n_inputs]
Y = XY.iloc[:N, -n_outputs:]

Optionally, save as txt-file which can be uploaded to e.g. [DataExplorer](https://buildingdesign.moe.dk/dataexplorer/).

In [ ]:
print('Saving to: ' + my_path + filename_monte_carlo[:-4] + 'csv')
# XY.to_csv(my_path + filename_monte_carlo[:-4] + 'csv', sep='\t', index=False)

# Sensitivity Analysis

Sensitivity analysis is performed using the *TOM* method which estimates the inputs' *total effects* on the outputs. It means that the contribution from 'interaction effects' between e.g. input *X1* and *X2* will be included in the sensitivity measure for both inputs. Thus it can be used in a *factor fixing* context where the object is to identify and potentially fix non-influential inputs (see [Global Sensitivity Analysis: The Primer](http://www.andreasaltelli.eu/file/repository/A_Saltelli_Marco_Ratto_Terry_Andres_Francesca_Campolongo_Jessica_Cariboni_Debora_Gatelli_Michaela_Saisana_Stefano_Tarantola_Global_Sensitivity_Analysis_The_Primer_Wiley_Interscience_2008_.pdf) for sensitivity analysis theory).

The *TOM* method may be applied to all outputs simulationeously as means to order the inputs by the overall influence on all outputs. This is makes it easier to observe changes of the overall most important inputs when exploring the data using [DataExplorer](https://buildingdesign.moe.dk/dataexplorer/). It does not simply asign equal weights to all outputs but instead reduce weights to highly correlated outputs. 

A detailed description of the *TOM* method is available in this [conference paper](https://vbn.aau.dk/da/publications/interactive-building-design-space-exploration-using-regionalized-).

## Single output

Use dropdown to specify which output should be addressed in the sensitivity analysis.

In [ ]:
output_labels = list(Y.columns)
dropdown = Dropdown(options= ['All'] + list(output_labels), description='Output(s):')
dropdown

Set sensitivy arguments and perform analysis.  
For large datasets, start with a small J value (e.g. 30) for a single output and see if the SA estimates have converged. If not, increase J until they converge. 

In [ ]:
J = 30              # Number random splits. Increase if values have not converged
add_dummy = True    # Specify if a 'dummy' variable should be added

if dropdown.value == 'All':
  print('Performing sensitivity analysis for all outputs.')
  y = Y.copy()
else:
  print('Performing sensitivity analysis for: ' + dropdown.value)
  y = Y.iloc[:, output_labels.index(dropdown.value)]

tom = sa.TOM(X, y, J=J, verbose=True, dummy=add_dummy)

`tom.KS_df` contains the J-averaged Kolmogorov-Smirnov distances and can be used to make convergence plots (in relation to J).  
In the `df_SA` DataFrame sensitivity measures are rescaled to sum to 100%. Both DataFrames can be used to make your own SA figures.

In [ ]:
df_SA = pd.DataFrame(columns=['Input', 'SA, tom'])
SA_score = tom.KS_df.iloc[-1,:].values # The last J'th averaged sample is used
SA_score = np.array([(val / sum(SA_score)) * 100 for val in SA_score])
for i, (col, score) in enumerate(zip(tom.KS_df.columns, SA_score)):
  df_SA.loc[i] = [col, score]
df_SA.sort_values(by="SA, tom", ascending=False) 

Optionally, order input columns by their influence with respect to a given output (or all outputs). The most influential will then be placed closest to the output columns, which make analysis in [DataExplorer](https://buildingdesign.moe.dk/dataexplorer/) easier. 

In [ ]:
filename_export = 'sorted_XY'  # Define filename

XY_export = XY.copy()
sa_sorted_inputs = list(df_SA.drop(df_SA[df_SA['Input'] == 'Dummy'].index).sort_values(by="SA, tom", ascending=True)['Input'])
sorted_columns = sa_sorted_inputs + list(output_labels)
XY_export = XY_export[sorted_columns]

# Save as csv or xlsx
XY_export.to_csv(my_path + filename_export + '.csv', sep='\t', index=False)
# XY_export.to_excel(my_path + filename_export + '.xlsx', index=False)

## All outputs
Create a figure with histogram and SA bar plots for all outputs.

In [ ]:
# Enable figures to be return from function
%config InlineBackend.close_figures = False

N_samples = X.shape[0] # Optionally, reduce the number of samples for faster SA
res, fig = sa_multiple(X.iloc[:N_samples,:], 
                       Y.iloc[:N_samples,:], 
                       J=100, 
                       include_SA_all=False, 
                       sort_by='all',
                       figsize='auto', # (7,5)
                       )

Optionally, save the figure as jpg, png, or svg.

In [ ]:
SAVE_FIG = False
if SAVE_FIG:
  # fig.savefig(my_path + file_name_only + '_TOM_SA.svg')
  # fig.savefig(my_path + file_name_only + '_TOM_SA.png')  
  fig.savefig(my_path + file_name_only + '_TOM_SA.jpg', dpi=300)

## External resources

- Free textbook on sensitivity analysis: [Global Sensitivity Analysis – The Primer](http://www.andreasaltelli.eu/file/repository/A_Saltelli_M_Ratto_T_Andres_F_Campolongo_J_Cariboni_D_Gatelli_M_Saisana_S_Tarantola_Global_Sensitivity_Analysis_The_Primer_Wiley_Interscience_2008_errata_corrige.pdf)
- [Conference paper](https://vbn.aau.dk/da/publications/interactive-building-design-space-exploration-using-regionalized-) on TOM and TOR methods
- [DataExplorer](https://buildingdesign.moe.dk/dataexplorer/) or [DesignExplorer](http://tt-acm.github.io/DesignExplorer/) for interactive design space exploration
- [SIMLAB & other software](https://ec.europa.eu/jrc/en/samo/simlab) from EU Science Hub
- UQlab & other software from [UQworld](https://uqworld.org/)
- [Online calculator](https://www.sobolindices.com/) of Sobol indices for datasets of max. 10 inputs and 500 samples (see [documentation](https://www.sobolindices.com/howto/))

# Machine Learning

## Choose output

Run below cell to make a dropdown to choose which output should be used in the supervised machine learning. Alternatively, all outputs can be included but accuracy will be lower for the individual outputs.

In [ ]:
output_labels = list(Y.columns)
dropdown_ml = Dropdown(options=['All (NN only)'] + list(output_labels), description='Output:')
dropdown_ml

## Prepare data

Split dataset into training set, validation set, and test set.

In [ ]:
VALID_SIZE = 0.25
TEST_SIZE = 0.25
shuffle_state = True
np.random.seed(42)

# Create a test set and a "full" training set to be split into training and validation sets
train_set_full, test_set = train_test_split(XY, test_size=TEST_SIZE, shuffle=shuffle_state)
train_set, valid_set = train_test_split(train_set_full, test_size=(VALID_SIZE / (1 - TEST_SIZE)), shuffle=shuffle_state)

total_sim = train_set.shape[0] + valid_set.shape[0] + test_set.shape[0]
display(f'Total of {total_sim} simulations. Length of data_raw: {len(XY)}.')
display(f'train set {train_set.shape}, valid set{valid_set.shape}, test set {test_set.shape}.')
display(f'Split ratios: {round((train_set.shape[0]/total_sim*100))} / {round((valid_set.shape[0]/total_sim*100))} / {round((test_set.shape[0]/total_sim*100))} %')

Split each set into input and output subsets and transform data for machine learning.

In [ ]:
x_train_full = train_set_full.iloc[:, :n_inputs]
x_train = train_set.iloc[:, :n_inputs]
x_valid = valid_set.iloc[:, :n_inputs]
x_test = test_set.iloc[:, :n_inputs]

if dropdown_ml.value[:3] == 'All':
  y_train_full = train_set_full.iloc[:, n_inputs:]
  y_train = train_set.iloc[:, n_inputs:]
  y_valid = valid_set.iloc[:, n_inputs:]
  y_test = test_set.iloc[:, n_inputs:]
  y_labels = train_set.iloc[:, n_inputs:].columns.to_list()
else:
  y_train_full = train_set_full[dropdown_ml.value].copy()
  y_train = train_set[dropdown_ml.value].copy()
  y_valid = valid_set[dropdown_ml.value].copy()
  y_test = test_set[dropdown_ml.value].copy()
  y_labels = [dropdown_ml.value]

num_pipeline = Pipeline([('standard_scaler', StandardScaler()),])
numerical_attr = x_train.select_dtypes(include=['float64', 'int64']).columns

full_pipeline = ColumnTransformer([
    ("num", num_pipeline, numerical_attr), 
    # ("ord", ordinal_pipeline, ord_attr),
    # ("cat", cat_pipeline, categorical_attr),
])

x_train_prepared = full_pipeline.fit_transform(x_train)
x_train_full_prepared = full_pipeline.fit_transform(x_train_full)
x_valid_prepared = full_pipeline.transform(x_valid)
x_test_prepared = full_pipeline.transform(x_test)

## Train models

### Neural network

#### Single model

Train a neural network *without* hyperparameter optimization. 

In [ ]:
# Define model parameters
model = MLPRegressor(hidden_layer_sizes=(20, 20), 
                  learning_rate_init=0.05, 
                  alpha=0.001, 
                  early_stopping=True, 
                  max_iter=500,  )

# Train neural network
model.fit(x_train_full_prepared, y_train_full.values)

# Make predictions
# (y_train_pred, y_train_full_pred, y_valid_pred, y_test_pred) = make_predictions(model);
y_train_pred = model.predict(x_train_prepared)
y_train_full_pred = model.predict(x_train_full_prepared)
y_valid_pred = model.predict(x_valid_prepared)
y_test_pred = model.predict(x_test_prepared)

# Clear output cell and show only R² values (averaged if multiple outputs)
clear_output() 
print(f'R² train/test: {r2_score(y_train_full, y_train_full_pred):.3f} / {r2_score(y_test, y_test_pred):.3f}')

See R²-values for each output.

In [ ]:
print(f'{" " * 18}TRAIN / TEST')
if y_train.ndim == 1:
  # print(f'R² train/test: {r2_score(y_train_full, y_train_full_pred):.3f} / {r2_score(y_test, y_test_pred):.3f} -> Output: "{y_train.to_frame().columns[0]}"')
  print(f'{y_train.to_frame().columns[0]:>15}   {r2_score(y_train_full, y_train_full_pred):.3f} / {r2_score(y_test, y_test_pred):.3f}')
else:
  for i in range(y_train_pred.shape[1]):
    weights = np.zeros(y_train_pred.shape[1])
    weights[i] = 1
    r2_train_i = r2_score(y_train_full, y_train_full_pred, multioutput=weights)
    r2_test_i = r2_score(y_test, y_test_pred, multioutput=weights)
    print(f'{y_train.columns[i]:>15}   {r2_train_i:.3f} / {r2_test_i:.3f}')

#### Grid-search optimization

Perform grid search to optimize hyperparameters. For more hyperparameters see [scikit-learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html).  
The number trained models equals the product of options for each hyperparameter multiplied by the number of cross validations (cv).

In [ ]:
cross_validations = 3

# Define hyperparameters to run grid search over
param_grid = [
              {'hidden_layer_sizes':[(20, 20), (50, 50), (20, 20, 20)], 
               'learning_rate_init':[0.001, 0.01, 0.03], 
              #  'learning_rate': ['constant', 'invscaling', 'adaptive'],
               'alpha':[0.001, 0.01], # default 0.001
               }
              ]

# Setup fixed hyperparameters
model = MLPRegressor(#alpha=0.001, # default 0.0001, 
                  early_stopping=True, max_iter=500, )

# Set grid search options
grid_search = GridSearchCV(model, param_grid, cv=cross_validations,
                          scoring='neg_mean_squared_error',
                          return_train_score=True, verbose=1, n_jobs=-1)

# Run grid search
grid_search.fit(x_train_full_prepared, y_train_full.values)

clear_output() # Clear output cell

# See cross validation results (RMSE) from grid search
cvres = grid_search.cv_results_ # Cross validation results from grid search
params_count = len(cvres['params'][0].keys()) # Number of hyperparameters
params_labels = list(cvres['params'][0].keys()) # Labels for hyperparameters
cvres_df = pd.DataFrame(columns=['RMSE'] + list(cvres['params'][0].keys()))
for i in range(len(cvres['mean_test_score'])):
  cvres_df = cvres_df.append({'RMSE': np.sqrt(-cvres['mean_test_score'][i]), **cvres['params'][i]}, ignore_index=True, )

# Performance of best model
model = grid_search.best_estimator_
# (y_train_pred, y_train_full_pred, y_valid_pred, y_test_pred) = make_predictions(model);
y_train_pred = model.predict(x_train_prepared)
y_train_full_pred = model.predict(x_train_full_prepared)
y_valid_pred = model.predict(x_valid_prepared)
y_test_pred = model.predict(x_test_prepared)
print(f'Best model:  {grid_search.best_params_} \nLowest RMSE: {cvres_df.RMSE.min():.3f}')
print(f'R² train/test: {r2_score(y_train_full, y_train_full_pred):.3f} / {r2_score(y_test, y_test_pred):.3f}\n')

display(cvres_df.sort_values(by='RMSE'))
cvres_df = ordinal_decode_cat_hyperparameters(cvres_df)

Perform SA on hyperparameters to see which have most influence on performance. This indicates which hyperparameters that require most attention and could be discretized further in additional grid searches. May not work if only two parameters have been varied with two discretizations.

In [ ]:
sa.TOM(cvres_df.drop('RMSE', axis=1), cvres_df.RMSE, J=100, dummy=False)

See hyperparameter distributions leading to the 20% best RMSE.  
This may reveal suitable values for the most important hyperparameters (among the considered parameters).

In [ ]:
cvres_df.loc[cvres_df.RMSE > cvres_df.RMSE.quantile(0.8)].drop('RMSE', axis=1).hist(
    bins=100, figsize=(cvres_df.shape[1]*3,2.5), layout=(1, cvres_df.shape[1]-1), grid=False);
cvres_df.loc[cvres_df.RMSE > cvres_df.RMSE.quantile(0.8)]

### Plot y-y scatter

Create a plot of true values, e.g. from Building Performance Simulations (BPS), and predicted values (ML estimates).

In [ ]:
fig, ax = plt.subplots()
# plt.scatter(y_train_pred, y_train, alpha=0.5, s=1, c='g')
# plt.scatter(y_train_pred, y_train, alpha=0.3, s=1)
plt.scatter(y_train_full, y_train_full_pred, alpha=0.4, s=5, c='g')
plt.scatter(y_test, y_test_pred, alpha=0.4, s=5, c='y')

plt.xlabel("BPS results")
plt.ylabel("ML estimates")

lims = [
    np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
    np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
]

ax.plot(lims, lims, 'r-', alpha=0.5, zorder=0)
ax.set_xlim(lims)
ax.set_ylim(lims)
ax.set_aspect('equal')

## New samples

Obtain probability densitity functions (PDF) for each column in an input sample matrix, X. Note that it is limited to discrete and continuous distributions.

In [ ]:
X_pdfs = obtain_pdfs(X, verbose=True)
# X_pdfs

In [ ]:
N_new = 10000

# Sample N new samples
X_new = create_new_samples(X_pdfs, N_new)

# Transform machine learning model
x_new_prep = full_pipeline.transform(X_new)
# x_new_prep = X_new # For Random Forest with no transforms

# Make predictions
y_new = model.predict(x_new_prep)

# Combine input values and predicted output values
X_new = pd.DataFrame(X_new, columns=X.columns)
Y_new = pd.DataFrame(y_new, columns=y_labels)
XY_new = pd.concat([X_new, Y_new], axis=1)

XY_new

See distributions of original and predicted outputs.

In [ ]:
Y_new_ = Y_new.copy()
Y_new_.columns = [s + '*' for s in Y_new.columns.to_list()]

YY_ = pd.concat([Y, Y_new_], axis=1)
YY_.hist(figsize=(Y.shape[1]*3, 5), bins=25, layout=(2, Y.shape[1]), grid=False);

Export predicted dataset to csv-file.

In [ ]:
SAVE_NEW_SAMPLES = True
if SAVE_NEW_SAMPLES:
  XY_new.to_csv(my_path + file_name_only + '_' + str(N_new) + '.csv', sep='\t', index=False)